In [1]:
!pip install --upgrade pip

In [2]:
!pip install pytesseract
!pip install opencv-python
!pip install python-multipart pillow pytesseract opencv-python numpy uvicorn

In [3]:
!source ../.venv/bin/activate

In [5]:
import pytesseract
import argparse
import cv2

In [6]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
	help="path to input image to be OCR'd")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] -i IMAGE
ipykernel_launcher.py: error: the following arguments are required: -i/--image


SystemExit: 2

/opt/homebrew/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
image = cv2.imread(args["image"])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# use Tesseract to OCR the image
text = pytesseract.image_to_string(image)
print(text)

NameError: name 'args' is not defined

In [3]:
import pytesseract
import argparse
import cv2

In [4]:
image_path = "./receipt.jpg"

In [24]:
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")

# Convert the image to RGB format
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Perform OCR
text = pytesseract.image_to_string(image)

In [12]:
cv2.imshow("nothing", image)
key = cv2.waitKey(0)
if key == ord('q') or key == 27:  # 27 is the ASCII code for 'ESC'
    cv2.destroyAllWindows()

In [13]:
def resize_receipt(image):
    height = 1800  # Target height for decent OCR
    aspect = image.shape[1] / image.shape[0]
    width = int(height * aspect)
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_CUBIC)

In [14]:
def enhance_receipt(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    
    # Remove shadows using morphological operations
    dilated = cv2.dilate(enhanced, np.ones((7,7), np.uint8))
    bg = cv2.medianBlur(dilated, 21)
    diff = 255 - cv2.absdiff(enhanced, bg)
    
    # Normalize
    normalized = cv2.normalize(diff, None, alpha=0, beta=255, 
                             norm_type=cv2.NORM_MINMAX)
    return normalized

In [15]:
def threshold_receipt(image):
    # Use Otsu's thresholding after Gaussian filtering
    blur = cv2.GaussianBlur(image, (5,5), 0)
    _, binary = cv2.threshold(blur, 0, 255, 
                            cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Additional processing for thermal paper
    kernel = np.ones((2,2), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    return binary

In [16]:
def deskew_receipt(image):
    # Find all non-zero points
    coords = np.column_stack(np.where(image > 0))
    
    # Get angle from minAreaRect
    angle = cv2.minAreaRect(coords)[-1]
    
    # Adjust angle
    if angle < -45:
        angle = 90 + angle
    
    # Rotate image
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h),
                            flags=cv2.INTER_CUBIC,
                            borderMode=cv2.BORDER_REPLICATE)
    return rotated

In [17]:
def preprocess_receipt(image):
    # Resize
    resized = resize_receipt(image)
    
    # Enhance and remove shadows
    enhanced = enhance_receipt(resized)
    
    # Threshold
    binary = threshold_receipt(enhanced)
    
    # Deskew
    deskewed = deskew_receipt(binary)
    
    # Final cleaning
    kernel = np.ones((2,2), np.uint8)
    cleaned = cv2.morphologyEx(deskewed, cv2.MORPH_CLOSE, kernel)
    
    return cleaned

In [18]:
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")

# Convert the image to RGB format
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Perform OCR
text = pytesseract.image_to_string(image)

In [19]:
text

'= a =i |\nSees Gt zZzie\nee eS rr OT Oe\n\nees sean com\nWWW. thaivillageplmince\n\n— — wa\n\n- 11/15/24 oy ST: 3\n4 DINE IN 7 (DINE IN)\na Ce a mnie es a 7.95\n1 Thai Clear Pork $\nVegetable Soup eo\n2 Thai Drunken Noodle $31...\n(Chicken)\n-Mild ey\n1 Thai Drunken Noodle $15.75\n(Chicken)\n-Medium Spicy ee nord\nTax: $3.86\nTotal:\n\n$59.06\n“* Unpaid ***\nNOTES: Dear Loyal Customers! We\nare participating Cash Intensive\n\nProgram. All menu prices are\nDiscounted Cash Price.\n\nRegular price\nincludes a small 4% convenience fee\n[o cover payment transaction fees.\nThank You for Supporting omall\n\nBusiness!\nYour feedback ig very il\n\né Let us k}\n\nNporttant to us.\nMail: thai\n\n1OW how we did.\nVillagaen@mamail ann.\n'

In [22]:
def clean_receipt_text(text):
    # Remove excessive spaces and normalize newlines
    text = ' '.join(text.split())
    
    # Basic cleaning
    text = re.sub(r'[=|]', '', text)  # Remove common OCR artifacts
    text = re.sub(r'\s+', ' ', text)  # Normalize spaces
    
    # Split into lines and remove empty ones
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    
    # Join back with single newlines
    cleaned_text = '\n'.join(lines)
    
    return cleaned_text

In [23]:
clean_receipt_text(text

'a i Sees Gt zZzie ee eS rr OT Oe ees sean com WWW. thaivillageplmince — — wa - 11/15/24 oy ST: 3 4 DINE IN 7 (DINE IN) a Ce a mnie es a 7.95 1 Thai Clear Pork $ Vegetable Soup eo 2 Thai Drunken Noodle $31... (Chicken) -Mild ey 1 Thai Drunken Noodle $15.75 (Chicken) -Medium Spicy ee nord Tax: $3.86 Total: $59.06 “* Unpaid *** NOTES: Dear Loyal Customers! We are participating Cash Intensive Program. All menu prices are Discounted Cash Price. Regular price includes a small 4% convenience fee [o cover payment transaction fees. Thank You for Supporting omall Business! Your feedback ig very il é Let us k} Nporttant to us. Mail: thai 1OW how we did. Villagaen@mamail ann.'

In [25]:
clean_receipt_text(text)

'a i Sees Gt zZzie ee eS rr OT Oe ees sean com WWW. thaivillageplmince — — wa - 11/15/24 oy ST: 3 4 DINE IN 7 (DINE IN) a Ce a mnie es a 7.95 1 Thai Clear Pork $ Vegetable Soup eo 2 Thai Drunken Noodle $31... (Chicken) -Mild ey 1 Thai Drunken Noodle $15.75 (Chicken) -Medium Spicy ee nord Tax: $3.86 Total: $59.06 “* Unpaid *** NOTES: Dear Loyal Customers! We are participating Cash Intensive Program. All menu prices are Discounted Cash Price. Regular price includes a small 4% convenience fee [o cover payment transaction fees. Thank You for Supporting omall Business! Your feedback ig very il é Let us k} Nporttant to us. Mail: thai 1OW how we did. Villagaen@mamail ann.'